In [204]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import boto3
import os
from sagemaker.amazon.amazon_estimator import get_image_uri
import sagemaker
from sagemaker import get_execution_role
from sklearn.model_selection import train_test_split
import numpy as np

import sagemaker
from random import shuffle
import multiprocessing
from multiprocessing import Pool
import csv
import nltk
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

In [205]:
data = pd.read_csv('../Data/full.csv');

train, validation, test = np.split(data.sample(frac=1, random_state=1729), [int(0.7 * len(data)), int(0.9*len(data))])



#train.to_csv('../Data/train.csv', sep=',')
#test.to_csv('../Data/test.csv', sep=',')


In [206]:
#train = pd.read_csv('../Data/train.csv')
#test = pd.read_csv('../Data/test.csv')

train.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
4734074,332,CASH_OUT,133853.35,C913507260,0.0,0.00,C1540196676,290239.96,424093.32,0,0
3114157,236,PAYMENT,11753.05,C1826965934,358115.0,346361.95,M1135314828,0.00,0.00,0,0
4914186,349,TRANSFER,3662944.04,C2058851291,0.0,0.00,C540952107,3801547.66,7464491.70,0,0
6291553,664,CASH_OUT,94020.02,C1134688769,214968.0,120947.98,C433697726,0.00,94020.02,0,0
4391453,321,PAYMENT,22714.83,C527730183,20626.0,0.00,M378158807,0.00,0.00,0,0


In [207]:
keep_list = ["step", "type", "amount", "oldbalanceOrg", "newbalanceOrig", "oldbalanceDest", "newbalanceDest", "isFraud", "isFlaggedFraud"]
train = train[keep_list]
test = test[keep_list]

train = pd.get_dummies(train, columns=["type"])
test = pd.get_dummies(test, columns=["type"])

In [208]:
train.head()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
4734074,332,133853.35,0.0,0.00,290239.96,424093.32,0,0,0,1,0,0,0
3114157,236,11753.05,358115.0,346361.95,0.00,0.00,0,0,0,0,0,1,0
4914186,349,3662944.04,0.0,0.00,3801547.66,7464491.70,0,0,0,0,0,0,1
6291553,664,94020.02,214968.0,120947.98,0.00,94020.02,0,0,0,1,0,0,0
4391453,321,22714.83,20626.0,0.00,0.00,0.00,0,0,0,0,0,1,0


In [209]:
test.head()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
649918,35,132520.80,11508.00,144028.80,0.00,0.00,0,0,1,0,0,0,0
6078014,518,5927.64,58852.27,52924.63,0.00,0.00,0,0,0,0,0,1,0
3352247,253,2899.46,6054.00,3154.54,964608.59,967508.05,0,0,0,1,0,0,0
2875453,227,193469.02,0.00,0.00,248357.55,441826.56,0,0,0,1,0,0,0
5729113,399,93088.86,107246.00,14157.14,0.00,93088.86,0,0,0,1,0,0,0


In [210]:
test.shape

(636262, 13)

In [211]:
train_labels = np.array(train["isFraud"]).astype("float32")

In [212]:
train_features = np.array(train.drop("isFraud", axis=1)).astype("float32")

In [213]:

test_labels = np.array(test["isFraud"]).astype("float32")
test_features  = np.array(test.drop("isFraud", axis=1)).astype("float32")
#pd.get_dummies(model_df, columns=["Block"])

In [214]:
train_features.shape

(4453834, 12)

In [215]:
def get_base_estimator(clf, sess, role):

    container = get_image_uri(boto3.Session().region_name, clf)

    est = sagemaker.estimator.Estimator(container,
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, clf),
                                    sagemaker_session=sess)
    return est

In [216]:
def get_estimator(clf, sess, role):
    
    container = get_image_uri(boto3.Session().region_name, clf)

    
    if clf == 'xgboost':
        est = get_base_estimator(clf, sess, role)
        est.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        num_round=100)
        
    elif clf == 'linear-learner':
        
        est = sagemaker.LinearLearner(role=sagemaker.get_execution_role(),
                                               train_instance_count=1,
                                               train_instance_type='ml.m4.xlarge',
                                               predictor_type='binary_classifier',
                                               num_classes=2)

    elif clf == 'knn':
        est = sagemaker.KNN(role=sagemaker.get_execution_role(),
                                              k = 10,
                                               train_instance_count=1,
                                               train_instance_type='ml.m4.xlarge',
                                               predictor_type='classifier',
                                                sample_size = 200)
        

        
        
    elif clf == 'factorization-machines':
        est = sagemaker.FactorizationMachines(role=sagemaker.get_execution_role(),
                                               train_instance_count=1,
                                               train_instance_type='ml.m4.xlarge',
                                               predictor_type='binary_classifier',
                                                num_factors = 2)
        
        
    return est

In [224]:
# add k-fold cross validation here 
sess = sagemaker.Session()
role = get_execution_role()
client = boto3.client('sagemaker')
bucket = sess.default_bucket()

s3_input_train = sagemaker.s3_input(s3_data='s3://{}/train/'.format(bucket), content_type='csv')
s3_input_test = sagemaker.s3_input(s3_data='s3://{}/test/'.format(bucket), content_type='csv')

In [225]:
import sagemaker
from sagemaker.amazon.amazon_estimator import RecordSet
import boto3

# instantiate the LinearLearner estimator object
multiclass_estimator = sagemaker.LinearLearner(role=sagemaker.get_execution_role(),
                                               train_instance_count=1,
                                               train_instance_type='ml.m4.xlarge',
                                               predictor_type='binary_classifier',
                                               num_classes=2)



In [ ]:
# wrap data in RecordSet objects
train_records = multiclass_estimator.record_set(train_features, train_labels, channel='train')
test_records = multiclass_estimator.record_set(test_features, test_labels, channel='test')

# start a training job
multiclass_estimator.fit([train_records, test_records])

2019-06-12 15:07:35 Starting - Starting the training job...
2019-06-12 15:07:39 Starting - Launching requested ML instances......
2019-06-12 15:08:45 Starting - Preparing the instances for training......
2019-06-12 15:09:46 Downloading - Downloading input data.....
Docker entrypoint called with argument(s): train
[06/12/2019 15:10:47 INFO 139828466566976] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/default-input.json: {u'loss_insensitivity': u'0.01', u'epochs': u'15', u'init_bias': u'0.0', u'lr_scheduler_factor': u'auto', u'num_calibration_samples': u'10000000', u'accuracy_top_k': u'3', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'num_point_for_scaler': u'10000', u'_log_level': u'info', u'quantile': u'0.5', u'bias_lr_mult': u'auto', u'lr_scheduler_step': u'auto', u'init_method': u'uniform', u'init_sigma': u'0.01', u'lr_scheduler_minimum_lr': u'auto', u'target_recall': u'0.8', u'num_models': u'auto', u'early_stopping_patience': u'3', u'mo

In [ ]:
def get_tuner(clf, est):
        
    if clf == 'xgboost':
        objective_metric_name = 'validation:auc'

        hyperparameter_ranges = {'eta': ContinuousParameter(0, 1),
                        'min_child_weight': ContinuousParameter(1, 10),
                        'alpha': ContinuousParameter(0, 2),
                        'max_depth': IntegerParameter(1, 10)}
        
    elif clf == 'knn':
        
        objective_metric_name = 'test:accuracy'

        hyperparameter_ranges = {'k': IntegerParameter(1, 1024),
                        'sample_size': IntegerParameter(256, 20000000)}
        
    elif clf == 'linear-learner':
        objective_metric_name = 'test:recall'
        
        hyperparameter_ranges = {'l1': ContinuousParameter(0.0000001,1),
                            'use_bias': CategoricalParameter([True, False])}
        
    elif clf == 'factorization-machines':
        objective_metric_name = 'test:binary_classification_accuracy'
        
        hyperparameter_ranges = {'bias_wd': IntegerParameter(1, 1000)}
        
    tuner = HyperparameterTuner(est,
                    objective_metric_name,
                    hyperparameter_ranges,
                    max_jobs=30,
                    max_parallel_jobs=3)
    
    return tuner

In [ ]:
def run_training_job(clf):

    # build the estimator
    est = get_estimator(clf, sess, role)

    # get the hyperparameter tuner config 
    # set this to look for recall somehow 
    if clf == 'xgboost':
        
        tuner = get_tuner(clf, est)
        
        tuner.fit({'train': s3_input_train, 'test': s3_input_test}) 

    else:
        # set the records
        train_records = est.record_set(train_features, train_labels, channel='train')
        test_records = est.record_set(test_features, test_labels, channel='validation')

        tuner = get_tuner(clf, est)
        
        tuner.fit([train_records, test_records])
    

In [ ]:
def magic_loop(models_to_run):
    pool = Pool(processes=multiprocessing.cpu_count())
    transformed_rows = pool.map(run_training_job, models_to_run)
    pool.close() 
    pool.join()

In [ ]:
clfs = ['xgboost', 'linear-learner', 'factorization-machines', 'knn']
magic_loop(clfs)